In [388]:
import json
import pandas as pd
import numpy as np

from collections import Counter
from ast import literal_eval

from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression

# Регрессия на категориальных признаках

In [389]:
df = pd.read_csv("vacancies.csv")

In [390]:
df.head()

,Unnamed: 0,accept_handicapped,postpoc_branded_desc,accept_kids,max_final_salary,max_salary,name,id,postpoc_desc,employment.name,salary.from_final,salary.currency,experience.name,schedule.name,employer.name,key_skills,address.city,salary.to_final
0,0,False,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n1 00...,False,NaN,NaN,Продавец-эксперт,74106345,Мебельная компания №1 в России в связи с актив...,Полная занятость,80000.0,RUR,Нет опыта,Сменный график,Много Мебели,NaN,NaN,NaN
1,1,False,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nМенеджер п...,False,NaN,-1.0,Менеджер по продажам в банке (м. Солнцево),74093467,Менеджер по продажам в банк: Райффайзенбанк — ...,Полная занятость,NaN,NaN,От 1 года до 3 лет,Полный день,Райффайзен Банк,"['Ориентация на результат', 'Работа в команде'...",Москва,NaN
2,2,False,NaN,False,20000.0,20000.0,Менеджер wildberries,74060491,Обязанности: Ведение аналитики личного кабине...,Полная занятость,20000.0,RUR,От 1 года до 3 лет,Гибкий график,Харитонов Антон Алексеевич,"['Амбициозность', 'Желание обучаться', 'Умение...",NaN,20000.0
3,3,False,NaN,False,34800.0,40000.0,Электромонтер по ремонту и обслуживанию электр...,74083801,Обязанности: Обслуживать силовые и осветитель...,Полная занятость,NaN,RUR,От 1 года до 3 лет,Полный день,ДОДХИБИМР,"['Пользователь ПК', 'Деловое общение', 'оформл...",село Лучинское,34800.0
4,4,False,NaN,False,50000.0,50000.0,Мастер по организации эксплуатации и ремонту з...,74079460,Обязанности: Обеспечение обслуживания и содер...,Полная занятость,50000.0,RUR,От 1 года до 3 лет,Полный день,АУ УР СК Чекерил,"['Пользователь ПК', 'Ремонтные работы', 'Орган...",Ижевск,50000.0


In [391]:
df_salaries = df[~df.max_final_salary.isnull()]

In [392]:
df_salaries.head()

,Unnamed: 0,accept_handicapped,postpoc_branded_desc,accept_kids,max_final_salary,max_salary,name,id,postpoc_desc,employment.name,salary.from_final,salary.currency,experience.name,schedule.name,employer.name,key_skills,address.city,salary.to_final
2,2,False,NaN,False,20000.0,20000.0,Менеджер wildberries,74060491,Обязанности: Ведение аналитики личного кабине...,Полная занятость,20000.0,RUR,От 1 года до 3 лет,Гибкий график,Харитонов Антон Алексеевич,"['Амбициозность', 'Желание обучаться', 'Умение...",NaN,20000.0
3,3,False,NaN,False,34800.0,40000.0,Электромонтер по ремонту и обслуживанию электр...,74083801,Обязанности: Обслуживать силовые и осветитель...,Полная занятость,NaN,RUR,От 1 года до 3 лет,Полный день,ДОДХИБИМР,"['Пользователь ПК', 'Деловое общение', 'оформл...",село Лучинское,34800.0
4,4,False,NaN,False,50000.0,50000.0,Мастер по организации эксплуатации и ремонту з...,74079460,Обязанности: Обеспечение обслуживания и содер...,Полная занятость,50000.0,RUR,От 1 года до 3 лет,Полный день,АУ УР СК Чекерил,"['Пользователь ПК', 'Ремонтные работы', 'Орган...",Ижевск,50000.0
9,9,False,NaN,False,34800.0,40000.0,Продавец-консультант,74078085,В сеть магазинов профессиональной косметики тр...,Полная занятость,34800.0,RUR,Нет опыта,Полный день,Эстель-Томск,"['Обслуживание покупателей', 'Розничная торгов...",Томск,34800.0
13,13,False,NaN,False,15660.0,18000.0,Слесарь-сантехник,74080096,Обязанности: Обслуживание санитарно -техничес...,Полная занятость,15660.0,RUR,От 1 года до 3 лет,Полный день,ФГБНУ ФАНЦ Юго-Востока,['Высокая ответственность'],Саратов,15660.0


In [393]:
def serialize(skills):
    if isinstance(skills, float):
        return []
    skills = skills.replace('"', "").replace("'", '"')
    return json.loads(skills)

In [394]:
df_salaries.key_skills = df_salaries.key_skills.apply(serialize)

/home/ephobia/.local/lib/python3.6/site-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [395]:
df_salaries.key_skills.apply(len).describe()

count    13834.000000
mean         4.239844
std          3.876316
min          0.000000
25%          1.000000
50%          4.000000
75%          5.000000
max         30.000000
Name: key_skills, dtype: float64

## Обработка навыков

1. Считаем самые частотные
2. Даем отдельные лейблы самым частотным, менее частотные убираем под один лейбл
3. Режем список навыков до 5

In [396]:
skill_set = []

for row in df_salaries.key_skills:
    skill_set +=  row

In [397]:
skill_counter = Counter(skill_set)

In [398]:
len(skill_counter)

4234

In [399]:
common_skills = {skill[0]: i for i, skill in enumerate(skill_counter.most_common(150))}

In [400]:
def filter_out_uncommon_skills(skills):
    final_skills = []
    for skill in skills:
        if skill in common_skills:
            final_skills.append(common_skills[skill])
        else:
            final_skills.append(150)
    final_skills.sort()
    if len(final_skills) < 10:
         final_skills = final_skills + (10 - len(final_skills)) * [151]
    else:
        final_skills = final_skills[:10]
    return np.asarray(final_skills)

In [401]:
df_salaries.vec_key_skills = df_salaries.key_skills.apply(filter_out_uncommon_skills)

/home/ephobia/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.


In [402]:
df_salaries.vec_key_skills.apply(len).describe()

count    13834.0
mean        10.0
std          0.0
min         10.0
25%         10.0
50%         10.0
75%         10.0
max         10.0
Name: key_skills, dtype: float64

In [403]:
skill_df = pd.DataFrame(np.array(list(df_salaries.vec_key_skills.values)),
                        columns=[f"skill_{i}" for i in range(10)],
                        index=df_salaries.index)

In [404]:
salary_df_skills = pd.concat([df_salaries, skill_df], axis=1)

## Обработка локаций

In [405]:
salary_df_skills["address.city"] = salary_df_skills["address.city"].apply(
    lambda x:x.lower() if not isinstance(x, float) else x
)

In [406]:
city_counter = Counter(salary_df_skills["address.city"].values)

In [407]:
len(city_counter)

929

In [408]:
common_cities = {city[0]: i for i, city in enumerate(city_counter.most_common(100))}

In [409]:
def filter_out_uncommon_cities(city):
    if city in common_cities:
        return common_cities[city]
    else:
        return 100

In [410]:
salary_df_skills["city_vec"] = salary_df_skills["address.city"].apply(filter_out_uncommon_cities)

## Обучение

In [420]:
cat_features = ["accept_handicapped", 
                "accept_kids", 
                "employment.name", 
                "experience.name",
                "schedule.name", 
                "city_vec", 
                "skill_0", "skill_1", "skill_2", 
                "skill_3", "skill_4", "skill_5", "skill_6", "skill_7", "skill_8", "skill_9"]

In [421]:
X = salary_df_skills[cat_features]

In [422]:
y = salary_df_skills.max_final_salary

In [423]:
enc = OrdinalEncoder()
X = enc.fit_transform(X)

In [424]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [425]:
regr = RandomForestRegressor(random_state=42)
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)

In [426]:
mean_absolute_error(y_test, y_pred)

28486.77680882078

In [427]:
d = {cat_features[i]: regr.feature_importances_[i] for i in range(len(regr.feature_importances_))}
Counter(d).most_common(20)

[('schedule.name', 0.19708929454127533),
 ('skill_0', 0.15670185888799282),
 ('city_vec', 0.13791366999446017),
 ('experience.name', 0.11932150573577678),
 ('skill_1', 0.07656906509263087),
 ('skill_2', 0.07354279313830737),
 ('skill_3', 0.055871125995338514),
 ('skill_4', 0.05347680480131649),
 ('skill_5', 0.03575065724396876),
 ('skill_7', 0.02053353942360616),
 ('skill_6', 0.019805544318278426),
 ('employment.name', 0.018857963761678812),
 ('accept_handicapped', 0.012353929178852722),
 ('skill_8', 0.011598904481060112),
 ('skill_9', 0.008503241152092406),
 ('accept_kids', 0.0021101022533643026)]

### Убрать навыки?

In [351]:
cat_features = ["accept_handicapped", 
                "accept_kids", 
                "employment.name", 
                "experience.name",
                "schedule.name", 
                "city_vec"]

In [352]:
X = salary_df_skills[cat_features]
y = salary_df_skills.max_final_salary
enc = OrdinalEncoder()
X = enc.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

regr = RandomForestRegressor(random_state=42)
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)

mean_absolute_error(y_test, y_pred)

33505.39467424036

# Регрессия на текстовых признаках

In [353]:
df = pd.read_csv("text_data.csv")

In [354]:
df.head()

,Unnamed: 0,accept_handicapped,postpoc_branded_desc,accept_kids,max_final_salary,max_salary,name,id,postpoc_desc,employment.name,...,address.city,salary.to_final,branded_num_sym,num_sym,branded_tokens,tokens,num_branded_tokens,num_tokens,branded_lemmas,lemmas
0,0,False,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n1 00...,False,NaN,NaN,Продавец-эксперт,74106345,Мебельная компания №1 в России в связи с актив...,Полная занятость,...,не указано,NaN,1078,610,"['1', '000', 'мебельных', 'салонов', '10', '00...","['Мебельная', 'компания', '№1', 'в', 'России',...",116,94,"['1', '000', 'мебельный', 'салон', '10', '000'...","['мебельный', 'компания', '№1', 'россия', 'свя..."
1,1,False,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nМенеджер п...,False,NaN,-1.0,Менеджер по продажам в банке (м. Солнцево),74093467,Менеджер по продажам в банк: Райффайзенбанк — ...,Полная занятость,...,Москва,NaN,2166,1848,"['Менеджер', 'по', 'продажам', 'в', 'банк:', '...","['Менеджер', 'по', 'продажам', 'в', 'банк:', '...",268,238,"['менеджер', 'продажа', 'банк', 'райффайзенбан...","['менеджер', 'продажа', 'банк', 'райффайзенбан..."
2,2,False,NaN,False,20000.0,20000.0,Менеджер wildberries,74060491,Обязанности: Ведение аналитики личного кабине...,Полная занятость,...,не указано,20000.0,0,314,[],"['Обязанности:', 'Ведение', 'аналитики', 'личн...",0,42,[],"['обязанности', 'ведение', 'аналитик', 'личный..."
3,3,False,NaN,False,34800.0,40000.0,Электромонтер по ремонту и обслуживанию электр...,74083801,Обязанности: Обслуживать силовые и осветитель...,Полная занятость,...,село Лучинское,34800.0,0,2465,[],"['Обязанности:', 'Обслуживать', 'силовые', 'и'...",0,256,[],"['обязанности', 'обслуживать', 'силовой', 'осв..."
4,4,False,NaN,False,50000.0,50000.0,Мастер по организации эксплуатации и ремонту з...,74079460,Обязанности: Обеспечение обслуживания и содер...,Полная занятость,...,Ижевск,50000.0,0,733,[],"['Обязанности:', 'Обеспечение', 'обслуживания'...",0,86,[],"['обязанности', 'обеспечение', 'обслуживание',..."


In [355]:
df_salaries = df[~df.max_final_salary.isnull()]

In [356]:
X = df_salaries.lemmas.apply(literal_eval)
y = df_salaries.max_final_salary.values

In [357]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [358]:
def identity_tokenizer(text):
    return text

tfidf = TfidfVectorizer(tokenizer=identity_tokenizer, lowercase=False)
X_train_vec = tfidf.fit_transform(X_train)
X_test_vec = tfidf.transform(X_test)

In [359]:
regr = LinearRegression()

In [360]:
regr.fit(X_train_vec, y_train)
y_pred = regr.predict(X_test_vec)

In [361]:
mean_absolute_error(y_test, y_pred)

59233.90660263326